In [1]:
from typing import Dict
import threading

import gym
import numpy as np
import torch

from stable_baselines3 import PPO as ALGO
from stable_baselines3.common.evaluation import evaluate_policy


/home/arnab/.miniconda3/envs/pydrl/lib/python3.7/site-packages/ale_py/roms/utils.py:90: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for external in metadata.entry_points().get(self.group, []):


In [2]:
# Hyper-Parameters
NUM_CLIENT_MODELS = 8
NUM_TRAINING_STEPS = 1
NUM_ITERATIONS = 1000000
UPDATE_FREQ = 10000
ENV_NAME = 'CartPole-v1'


In [3]:
global_model = ALGO.load('initial_d')

client_model_1 = ALGO(
    "MlpPolicy",
    gym.make(ENV_NAME),
    n_epochs=1,
    n_steps= 2
)

client_model_2 = ALGO(
    "MlpPolicy",
    gym.make(ENV_NAME),
    n_epochs=1,
    n_steps= 2
)


/home/arnab/.miniconda3/envs/pydrl/lib/python3.7/site-packages/stable_baselines3/ppo/ppo.py:138: UserWarning: You have specified a mini-batch size of 64, but because the `RolloutBuffer` is of size `n_steps * n_envs = 2`, after every 0 untruncated mini-batches, there will be a truncated mini-batch of size 2
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=2 and n_envs=1)
  f"You have specified a mini-batch size of {batch_size},"


In [4]:
def _eval(model, env, index, fitnesses):
    fitness, _ = evaluate_policy(model, env)
    fitnesses[index] = fitness


def evaluate(models, env, index, eval_results, message='', verbose=0):
    # print('Starting Eval')
    iterations = 10
    fitnesses = [0 for i in range(iterations)]
    eval_threads = []
    for i in range(iterations):
        thread = threading.Thread(target=_eval, args=(
            models[index], gym.make(ENV_NAME), i, fitnesses))
        eval_threads.append(thread)

    # Start Threads
    for thread in eval_threads:
        thread.start()

    # Join Threads (wait until thread is completely executed)
    for thread in eval_threads:
        thread.join()

    mean_fitness = np.mean(sorted(fitnesses))
    eval_results[index] = mean_fitness
    print(f'Type {message} Mean reward: {mean_fitness}')


In [5]:
def train(models, index, timesteps):
    # print('Starting Training')
    models[index] = models[index].learn(
        reset_num_timesteps=True, total_timesteps=timesteps)
    # print('Completed Training')


In [6]:
client_model_1.set_parameters(global_model.get_parameters())
client_model_2.set_parameters(global_model.get_parameters())

# global_model.save('initial_d')

evaluate([global_model], gym.make(ENV_NAME), 0, [0])
evaluate([client_model_1], gym.make(ENV_NAME), 0, [0])
evaluate([client_model_2], gym.make(ENV_NAME), 0, [0])

/home/arnab/.miniconda3/envs/pydrl/lib/python3.7/site-packages/stable_baselines3/common/evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


Type  Mean reward: 9.64
Type  Mean reward: 9.35
Type  Mean reward: 9.35


In [7]:
for i in range(NUM_ITERATIONS):
    client_model_1.learn(reset_num_timesteps=True, total_timesteps=NUM_TRAINING_STEPS)
    client_model_2.learn(reset_num_timesteps=True, total_timesteps=NUM_TRAINING_STEPS)
    
    global_dict = global_model.policy.state_dict()
    # print(global_dict[])
    for param in client_model_1.policy.named_parameters():
        temp_tensor = torch.zeros(param[1].size())
        global_dict[param[0]].add_(temp_tensor.add_(-0.003, param[1].grad))
    
    for param in client_model_2.policy.named_parameters():
        temp_tensor = torch.zeros(param[1].size())
        global_dict[param[0]].add_(temp_tensor.add_(-0.003, param[1].grad))
    

    global_model.policy.load_state_dict(global_dict)
    client_model_1.policy.load_state_dict(global_dict)
    client_model_2.policy.load_state_dict(global_dict)

    if ((i+1) % UPDATE_FREQ) == 0:
        evaluate([global_model], gym.make(ENV_NAME), 0, [0])
        # evaluate([client_model_1], gym.make(ENV_NAME), 0, [0])
        # evaluate([client_model_2], gym.make(ENV_NAME), 0, [0])
        print('-------')
    # break


/home/arnab/.miniconda3/envs/pydrl/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  ../torch/csrc/utils/python_arg_parser.cpp:1025.)
  if __name__ == '__main__':


Type  Mean reward: 29.589999999999996
-------
Type  Mean reward: 82.28999999999999
-------
Type  Mean reward: 85.37
-------
Type  Mean reward: 84.17
-------
Type  Mean reward: 94.20000000000002
-------
Type  Mean reward: 89.89000000000001
-------
Type  Mean reward: 90.83000000000001
-------
Type  Mean reward: 94.08
-------
Type  Mean reward: 89.73
-------
Type  Mean reward: 88.85
-------
Type  Mean reward: 87.5
-------
Type  Mean reward: 92.54999999999998
-------
Type  Mean reward: 88.8
-------
Type  Mean reward: 91.58
-------
Type  Mean reward: 90.80000000000001
-------
